In [ ]:
pip install numpy==1.26.0 scipy==1.14.0 meshio==5.3.5 libigl==v2.5.1 polyscope==2.2.1 ilupp==1.0.2 ipctk==1.2.0 pbatoolkit

In [1]:
import pbatoolkit as pbat
import pbatoolkit.math.linalg
import igl
import polyscope as ps
import polyscope.imgui as imgui
import numpy as np
import scipy as sp
import argparse
import meshio
import os
import pathlib

# Define the input path for the triangle mesh

In [4]:
current_path = os.getcwd()

input_path = os.path.join(current_path, 'python' ,'examples', 'notebooks', 'resources', 'tetreahedron.obj')

In [5]:
imesh = meshio.read(input_path)
V, F = imesh.points, imesh.cells_dict["triangle"]
V, F = V.astype(np.float64, order='c'), F.astype(np.int64, order='c')

ValueError: len(points) = 220, but len(point_data["obj:vn"]) = 216

# Construct Galerkin laplacian, mass and gradient operators

In [ ]:
mesh = pbat.fem.Mesh(
    V.T, F.T, element=pbat.fem.Element.Triangle, order=1)
V, F = mesh.X.T, mesh.E.T
detJeL = pbat.fem.jacobian_determinants(mesh, quadrature_order=1)
GNeL = pbat.fem.shape_function_gradients(mesh, quadrature_order=1)
L = pbat.fem.Laplacian(
    mesh, detJeL, GNeL, quadrature_order=1).to_matrix()
detJeM = pbat.fem.jacobian_determinants(mesh, quadrature_order=2)
M = pbat.fem.MassMatrix(mesh, detJeM, dims=1,
                         quadrature_order=2).to_matrix()

# Setup heat diffusion

In [ ]:
dt = 0.016
c = 1
A = M - c*dt*L

# Precompute linear solversZ

In [ ]:
Ainv = pbat.math.linalg.ldlt(A)
Ainv.compute(A)

In [ ]:
ps.set_up_dir("z_up")
ps.set_front_dir("neg_y_front")
ps.set_ground_plane_mode("shadow_only")
ps.init()
vmm = ps.register_surface_mesh("model", V, F)
vmm.set_smooth_shade(True)
vms = ps.register_surface_mesh("smoothed", V, F)
vms.set_smooth_shade(True)
smooth = False

In [ ]:
def callback():
    global dt, Ainv, M, L, smooth, V, c
    dtchanged, dt = imgui.InputFloat("dt", dt)
    cchanged, c = imgui.SliderFloat("c", c, v_min=0, v_max=100)
    if dtchanged or cchanged:
        A = M - c*dt*L
        Ainv.factorize(A)
    _, smooth = imgui.Checkbox("smooth", smooth)
    if smooth:
        V = Ainv.solve(M @ V)
        vms.update_vertex_positions(V)

ps.set_user_callback(callback)
ps.show()